# Setup the Running Drive Folder



In [0]:
# Load the Drive helper and mount
from google.colab import drive

# Mount the drive folder. This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

# Opens the project folder. IMPORTANT: Change to your route
%cd 'drive/My Drive/TEC/Tesis/BBBC006x256/'

# Dependencies and Libraries



## Install Dependencies



In [2]:
!pip install scipy
!pip install torchvision

## Modules Import

In [0]:
import sys
import os
import numpy as np
#import cupy as cp
import random
import time
import math
import csv
import json
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch import argmax
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataset import ConcatDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms.functional as Ft
from torch.autograd.variable import Variable

from scipy.ndimage.morphology import distance_transform_edt as dist_trans

## Global Variables

Configure global variables such as the mean and std of the dataset you are going to use.

In [0]:
#For Original Dataset
globalMean = 0.0054;
globalStd = 0.0037;

# Misc Functions

Auxiliary code, used to save the network status, results and predictions

## Save Network Epoch Results



In [0]:
""" 
    Export data to csv format. Creates new file if doesn't exist,
    otherwise update it.
    Args:
        header (list): headers of the column
        value (list): values of correspoding column
        folder: folder path
        file_name: file name with path
"""
def export_history(header, value, folder, file_name):
    # If folder does not exists make folder
    if not os.path.exists(folder):
        os.makedirs(folder)

    file_path = folder + file_name
    file_existence = os.path.isfile(file_path)

    # If there is no file make file
    if file_existence == False:
        file = open(file_path, 'w', newline='')
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerow(value)
    # If there is file overwrite
    else:
        file = open(file_path, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow(value)
    # Close file when it is done with writing
    file.close()


""" 
    Computes and stores the average and current value
    Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
"""  
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Save Network State

In [0]:
""" 
    Save the state of a net.
"""
def save_checkpoint(state, path='checkpoint/', filename='weights.pth'):
    # If folder does not exists make folder
    if not os.path.exists(path):
        os.makedirs(path)

    filepath = os.path.join(path, filename)
    torch.save(state, filepath)

## Save Network Predictions

In [0]:
def to_image(tensor, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.

    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    tensor = tensor.cpu()
    grid = make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).numpy()
    im = Image.fromarray(ndarr)
    return im

In [0]:
def save_dataset_loaders(dir_save, load, n_channels, n_classes, loaders, conversion, out, full_loader=True):
  with torch.no_grad():

        print("""
        \nModel predictions are going to be saved with:
        Output Directory: {},
        Weights Directory: {},
        Number of Input Channels: {},
        Number of Classes: {},
        Conversion type: {},
        Out Function: {}
              """.format(dir_save, load, n_channels, n_classes, conversion, out))
        
        # Make sure that loaders is a list of loaders, if it is a single dataloader; pack into a list
        if isinstance(loaders, DataLoader):
          loaders = [loaders]

        # Use GPU or not
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")

        # Create the model - 256 classes are used so the resulting image can better reflect the grayscale groundtruth
        #net = UNet(n_channels=1, n_classes=1).to(device)
        # Create the model
        net = UNet(n_channels=n_channels, n_classes= n_classes).to(device)
        net = torch.nn.DataParallel(net, device_ids=list(
            range(torch.cuda.device_count()))).to(device)

        # Load old weights
        if load:
            net.load_state_dict(torch.load(load)["state_dict"])
            print('Model loaded from {}'.format(load))
            
        net.eval()
        
        # Full loader is a variable that basically works to check the type of loader we are receiving in this function
        # is true if the loader is the same as the one used in training (img, gt, id) or false; specially made to save predictions (img, id)
        # This is necessary as we already had a function that saved the dataset with a special loader.
        if full_loader:
          
          for loader in loaders:

            for batch_idx, (data, gt, id_n) in enumerate(loader):
   
              # Use GPU or not
              data = data.to(device, dtype=torch.float)
              
              # Forward
              predictions = net(data)
              
              if out == "Softmax":
                predictions = F.softmax(predictions)
              
              # Saves the image
              img = to_image(predictions)
              img_new = img.convert(conversion)
              img_new.save(dir_save + id_n[0])

              if batch_idx % 10 == 0:
                print('[{}/{} ({:.0f}%)]'.format(
                  batch_idx, len(loader), 100. * batch_idx / len(loader)))
            
        else:
        
          for loader in loaders:

            for batch_idx, (data, id_n) in enumerate(loader):
        
              # Use GPU or not
              data = data.to(device, dtype=torch.float)
              
              # Forward
              predictions = net(data)
              
              if out == "Softmax":
                predictions = F.softmax(predictions)
              
              # Saves the image
              img = to_image(predictions)
              img_new = img.convert(conversion)
              img_new.save(dir_save + id_n[0]) 

              if batch_idx % 10 == 0:
                print('[{}/{} ({:.0f}%)]'.format(
                  batch_idx, len(loader), 100. * batch_idx / len(loader)))
        

def save_dataset(dir_save, load, n_channels, n_classes, dir_img, conversion, out, back_load=None, two_inputs=None):
  loader = get_dataloader_transform(dir_img, back_load, two_inputs)
  save_dataset_loaders(dir_save, load, n_channels, n_classes, loader, conversion, out, False)
  
def save_dataset_k_fold(dir_save, load, n_channels, n_classes, dir_img, conversion, out, k, dir_gt, batch_size, current_iter, back_model_load=None, two_inputs=None):
  ids = get_partitions(k)
  back_loaders = get_loaders_kfolds(ids, k, dir_img, dir_gt, batch_size, current_iter, back_model_load, two_inputs)
  save_dataset_loaders(dir_save, load, n_channels, n_classes, back_loaders[1], conversion, out, True)    

In [0]:
def save_predictions(i, k, dir_predictions, results_top, results_back, dir_img, dir_gt, two_inputs):

  # Creates predictions' directory
  if not os.path.isdir(dir_predictions + "Fold-{}/".format(str(i))):
    os.makedirs(dir_predictions + "Fold-{}/".format(str(i)))

  if results_back is not None:
    back_load = results_back + "best_weights" + str(i) + ".pth"
  else:
    back_load = None
    two_inputs = None

  if two_inputs:
    top_n_channels = 2
  else:
    top_n_channels = 1
    
  # Saves the results for future metrics calculation
  save_dataset_k_fold(dir_save=dir_predictions + "Fold-{}/".format(str(i)), 
                load=results_top + "best_weights" + str(i) + ".pth",
                n_channels = top_n_channels, 
                n_classes=3, 
                dir_img=dir_img, 
                conversion='RGB', 
                out=None,
                k=k,
                dir_gt=dir_gt,
                batch_size = 1,
                current_iter = i,
                back_model_load=back_load,
                two_inputs=two_inputs)

# U-Net Architecture



In [0]:
""" 
    This file defines every layer (or group of layers) that are inside UNet.
    At the final the architecture UNet is defined as a conjuntion of the elements created.
"""
class double_conv(nn.Module):
    ''' Applies (conv => BN => ReLU) two times. '''

    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            # inplace is for aply ReLU to the original place, saving memory
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            # inplace is for aply ReLU to the original place, saving memory
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    ''' First Section of U-Net. '''

    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    ''' Applies a MaxPool with a Kernel of 2x2,
        then applies a double convolution pack. '''

    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()

        self.mpconv = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    ''' Applies a Deconvolution and then applies applies a double convolution pack. '''

    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        
        # Bilinear is used to save computational cost
        if bilinear:
            self.up = nn.Upsample(
                scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(
                in_ch//2, in_ch//2, kernel_size=2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(input=x2, pad=(diffX // 2, diffX // 2,
                                  diffY // 2, diffY // 2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    ''' Applies the last Convolution to give an answer. '''

    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()

        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        x = self.conv(x)
        return x

      

In [0]:
class UNet(nn.Module):
    ''' This Object defines the architecture of U-Net. '''

    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()

        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)
        
        
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        #x = F.softmax(x) # New softmax layer
        return x

# Dataset Handlders

The nexts cells are there for the loading process. In this case we use the Dataset and Dataloader objects given by Pytorch. In the training process we will call this section

## Datasets

In these cells we define the way our datasets will behave and how the data will be retrieved

In [0]:
'''
    Class that defines the reading and processing of the images.
    Specialized on BBBC006 dataset.
'''
class BBBCDataset(Dataset):
    def __init__(self, ids, dir_data, dir_gt, extension='.png'):

        self.dir_data = dir_data
        self.dir_gt = dir_gt
        self.extension = extension

        # Use GPU or not
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        # Transforms
        self.data_transforms = {
            'imgs': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([globalMean],[globalStd])
            ]),
            'masks': transforms.Compose([
                transforms.ToTensor()
            ]),
        }

        # Images IDS
        self.ids = ids

        # Calculate len of data
        self.data_len = len(self.ids)

    '''
        Ask for an image.
    '''
    def __getitem__(self, index):
        # Get an ID of a specific image
        id_img = self.dir_data + self.ids[index] + self.extension
        id_gt = self.dir_gt + self.ids[index] + self.extension
        # Open Image and GroundTruth
        img = Image.open(id_img).convert('L')
        gt = Image.open(id_gt)
        # Applies transformations
        img = self.data_transforms['imgs'](img)
        img = img.to(self.device)
#        img = img / torch.max(img)
        gt = self.data_transforms['masks'](gt)
        gt = gt.to(self.device)

        return (img, gt, self.ids[index]+self.extension)

    '''
        Length of the dataset.
        It's needed for the upper class.
    '''
    def __len__(self):
        return self.data_len


In [0]:
'''
    Class that defines the reading and processing of the images.
    Specialized on BBBC006 dataset.
'''
class BBBCDataset_Top(Dataset):
    def __init__(self, ids, dir_data, dir_gt, load, extension='.png'):

        self.dir_data = dir_data
        self.dir_gt = dir_gt
        self.extension = extension
        self.load = load

        # Use GPU or not
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        # Create the model
        # As this is the top model we know it will always have 1 channel of input and 1 class of output
        back_model = UNet(1, 1).to(self.device)
        self.back_model = torch.nn.DataParallel(back_model, device_ids=list(
        range(torch.cuda.device_count()))).to(self.device)

        #self.back_model.eval()

        # Load the weights
        self.back_model.load_state_dict(torch.load(load)['state_dict'])
        print('Model loaded from {}'.format(load))

        # Transforms
        self.data_transforms = {
            'imgs': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([globalMean],[globalStd])
            ]),
            'masks': transforms.Compose([
                transforms.ToTensor()
            ]),
            'preds': transforms.Compose([
                transforms.ToTensor()
            ]),
        }

        # Images IDS
        self.ids = ids

        # Calculate len of data
        self.data_len = len(self.ids)

    '''
        Ask for an image.
    '''
    def __getitem__(self, index):

        # Get an ID of a specific image
        id_img = self.dir_data + self.ids[index] + self.extension
        id_gt = self.dir_gt + self.ids[index] + self.extension
        # Open Image and GroundTruth
        img = Image.open(id_img).convert('L')
        gt = Image.open(id_gt)
        # Applies transformations
        img = self.data_transforms['imgs'](img)
        img = img.to(self.device)
        # Resize image to "batch" size so the weights work
        img = img[None,:,:,:]
#       img = img / torch.max(img)
        gt = self.data_transforms['masks'](gt)
        gt = gt.to(self.device)
        
        # Obtains the back_model prediction
        img_top = self.back_model(img)
        img_top = img_top[0,:,:,:]
        
        # Prediction to Greyscale
        #img_top = to_image(img_top)
        #img_top = img_top.convert('L')

        #img_top = self.data_transforms['preds'](img_top)
        img_top = img_top.to(self.device)
        #img_top = img_top / 255;

        return (img_top, gt, self.ids[index]+self.extension)

    '''
        Length of the dataset.
        It's needed for the upper class.
    '''
    def __len__(self):
        return self.data_len


In [0]:
'''
    Class that defines the reading and processing of the images.
    Specialized on BBBC006 dataset.
'''
class BBBCDataset_TwoInputs(Dataset):
    def __init__(self, ids, dir_data, dir_gt, load, extension='.png'):

        self.dir_data = dir_data
        self.dir_gt = dir_gt
        self.extension = extension
        self.load = load

        # Use GPU or not
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        # Create the model
        # As this is the top model we know it will always have 1 channel of input and 1 class of output
        back_model = UNet(1, 1).to(self.device)
        self.back_model = torch.nn.DataParallel(back_model, device_ids=list(
        range(torch.cuda.device_count()))).to(self.device)

        #self.back_model.eval()

        # Load the weights
        self.back_model.load_state_dict(torch.load(load)['state_dict'])
        print('Model loaded from {}'.format(load))

        # Transforms
        self.data_transforms = {
            'imgs': transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([globalMean],[globalStd])
            ]),
            'masks': transforms.Compose([
                transforms.ToTensor()
            ]),
            'preds': transforms.Compose([
                transforms.ToTensor()
            ]),
        }

        # Images IDS
        self.ids = ids

        # Calculate len of data
        self.data_len = len(self.ids)

    '''
        Ask for an image.
    '''
    def __getitem__(self, index):

        # Get an ID of a specific image
        id_img = self.dir_data + self.ids[index] + self.extension
        id_gt = self.dir_gt + self.ids[index] + self.extension
        # Open Image and GroundTruth
        img = Image.open(id_img).convert('L')
        gt = Image.open(id_gt)
        # Applies transformations
        img = self.data_transforms['imgs'](img)
        img = img.to(self.device)
        # Resize image to "batch" size so the weights work
        img_back = img[None,:,:,:]
#       img = img / torch.max(img)
        gt = self.data_transforms['masks'](gt)
        gt = gt.to(self.device)
        
        # Obtains the back_model prediction
        img_top = self.back_model(img_back)
        img_top = img_top[0,:,:,:]
        
        # Prediction to Greyscale
        #img_top = to_image(img_top)
        #img_top = img_top.convert('L')

        #img_top = self.data_transforms['preds'](img_top)
        #img_top = img_top.to(self.device)
        #img_top = img_top / 255;

        img_final = torch.cat((img,img_top),0)
        #img_final = img_final.to(self.device)

        return (img_final, gt, self.ids[index]+self.extension)

    '''
        Length of the dataset.
        It's needed for the upper class.
    '''
    def __len__(self):
        return self.data_len

## Dataloaders

In the next code cells we define some dataloaders to pull the data from the Dataset classes

In [0]:
# We assume the existance of a file kfolds_partitions.txt with each partition
def get_partitions(k):

  with open("kfolds_partitions.txt") as file_partitions:
    
    # Read the partitions as JSON
    json_file = json.load(file_partitions)

    ids_partitions = []

    for i in range(0, k):
      
      partition_i = []

      for id_val in json_file["partition_" + str(i)]:
        partition_i.append(str(id_val))

      ids_partitions.append(partition_i)

    return ids_partitions
    
  
# Function in charge of obtaining the training and validation loaders from 
# the ids partitioned with get_partitions
def get_loaders_kfolds(ids_partitions, k, dir_img, dir_gt, batch_size, current_iter, back_model_load, two_inputs):
  
  # Variable that holds the train partitions ids
  train_ids = []

  # Iterates over the ids, while the ids_set examined is not the validation one, then it is appended to the train ids
  for j in range(0, k):
    if j != current_iter:
      
      train_ids += ids_partitions[j]

  # Gets the datasets according to the kind of model we are training
  if back_model_load and two_inputs:
    val_dset = BBBCDataset_TwoInputs(ids=ids_partitions[current_iter], dir_data=dir_img, dir_gt=dir_gt, load=back_model_load)
    train_dset = BBBCDataset_TwoInputs(ids=train_ids, dir_data=dir_img, dir_gt=dir_gt, load=back_model_load)
  elif back_model_load:
    val_dset = BBBCDataset_Top(ids=ids_partitions[current_iter], dir_data=dir_img, dir_gt=dir_gt, load=back_model_load)
    train_dset = BBBCDataset_Top(ids=train_ids, dir_data=dir_img, dir_gt=dir_gt, load=back_model_load)
  else:
    val_dset = BBBCDataset(ids=ids_partitions[current_iter], dir_data=dir_img, dir_gt=dir_gt)
    train_dset = BBBCDataset(ids=train_ids, dir_data=dir_img, dir_gt=dir_gt)

  # Obtains the array of loaders we need to wrap both our datasets
  loaders = [
             DataLoader(train_dset, batch_size=batch_size, shuffle=True),
             DataLoader(val_dset, batch_size=batch_size, shuffle=True)
  ]

  return loaders

## Dataset Mean-Std

In [0]:
# Remember that the dataloader should have load without normalization
def getMeanStd(dataloader):
  # Use GPU or not
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  (std, mean) = (0, 0)
  for batch_idx, (data, gt, weights) in enumerate(dataloader):
    # Use GPU or not
    data, gt = data.to(device), gt.to(device)
    (tempStd, tempMean) = torch.std_mean(data)
    std = std + tempStd
    mean = mean + tempMean
  std = std / len(dataloader)
  mean = mean / len(dataloader)
  return (mean, std)

# Model Training & Validation

## Loss functions

In [0]:
"""
    Class that defines the Cross Entropy Loss Function
"""
class CELoss(nn.Module):
    def __init__(self):
        super(CELoss, self).__init__()

    def forward(self, y_pred, y_true):
        return -torch.mean(torch.sum(y_true*torch.log(F.softmax(y_pred,dim=1)),dim=1))

## Training & Validation Setup

In [0]:
""" 
    Functions that trains a net.
"""
def train_net(net, device, loader, optimizer, criterion, batch_size, loss_type):
    
    net.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    time_start = time.time()

    for batch_idx, (data, gt, weights) in enumerate(loader):

        # Use GPU or not
        data, gt = data.to(device), gt.to(device)

        # Forward
        predictions = net(data)
        
        # Loss Calculation
        if loss_type == "Cross":
            labels = argmax(gt, dim=1)
            loss = criterion(predictions, labels)
        else:
            loss = criterion(predictions, gt)
            
        # Updates the record
        train_loss.update(loss.item(), predictions.size(0))
        train_acc.update(-loss.item(), predictions.size(0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Calculates new parameters
        optimizer.step()

        print('[{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
            batch_idx * len(data), len(loader)*batch_size,
            100. * batch_idx / len(loader), loss.item()))

    time_dif = time.time() - time_start
    print('\nAverage Training Loss: ' + str(train_loss.avg))
    print('\nAverage Training Accuracy: ' + str(train_acc.avg))
    print('Train Time: It tooks %.4fs to finish the epoch.' % (time_dif))
            
    return train_loss.avg, train_acc.avg

In [0]:
""" 
    Function that validates the net.
"""
def val_net(net, device, loader, criterion, batch_size):
    net.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    time_start = time.time()
    with torch.no_grad():
        for batch_idx, (data, gt, weights) in enumerate(loader):

            # Use GPU or not
            data, gt = data.to(device), gt.to(device)

            # Forward
            predictions = net(data)
            
            # Loss Calculation
            loss = criterion(predictions, gt)

            # Updates the record
            val_loss.update(loss.item(), predictions.size(0))
            val_acc.update(-loss.item(), predictions.size(0))
            
            print('[{}/{} ({:.0f}%)]\t\tLoss: {:.6f}'.format(
                batch_idx * len(data), len(loader)*batch_size,
                100. * batch_idx / len(loader), loss.item()))
    
    time_dif = time.time() - time_start
    print('\nValidation set: Average loss: '+ str(val_loss.avg))
    print('\nAverage Validation Accuracy: ' + str(val_acc.avg))
    print('Validation time: It tooks %.4fs to finish the Validation.' % (time_dif))
    
    return val_loss.avg, val_acc.avg

In [0]:
'''
    Configure every aspect of the run.
    Runs the training and validation.
'''
def setup_and_run_train(n_channels, n_classes, dir_img, dir_gt, dir_results, load, 
                val_perc, batch_size, epoch_inicial, epochs, lr, run, optimizer, loss, loaders):
    
    # Use GPU or not
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # Create the model
    net = UNet(n_channels, n_classes).to(device)
    net = torch.nn.DataParallel(net, device_ids=list(
        range(torch.cuda.device_count()))).to(device)

    # Load old weights
    if load:
      try:
        net.load_state_dict(torch.load(load)['state_dict'])
        print('Model loaded from {}'.format(load))
      except Exception as e:
        print('The Model could not be loaded, random weights will be used: {}'.format(str(e)))
      try:
        results_csv = pd.read_csv(dir_results + "result{}.csv".format(str(run)))
        best_loss = np.min(results_csv["validation loss"])
        print('Best loss recorded in last training: {}'.format(best_loss))
      except Exception as e:
        print("Error cargando los datos del csv: {}".format(str(e)))
    else:
      best_loss = 10000

    train_loader, val_loader = loaders[0], loaders[1]
    train_size = len(loaders[0]) * batch_size
    loader_size = len(loaders[1]) * batch_size
            
    # Pretty print of the run
    print('''\n
    Starting training:
        Dataset: {}
        Num Channels: {}
        Groundtruth: {}
        Num Classes: {}
        Folder to save: {}
        Load previous: {}
        Training size: {}
        Validation size: {}
        Batch size: {}
        Epochs: {}
        Learning rate: {}
        Optimizer: {}
        Loss Function: {}
        CUDA: {}
    '''.format(dir_img, n_channels, dir_gt, n_classes, dir_results, load, 
            train_size, loader_size, batch_size, epochs, lr, optimizer, loss, use_cuda))

    # Definition of the optimizer ADD MORE IF YOU WANT
    if optimizer == "Adam":
        optimizer = torch.optim.Adam(net.parameters(),
                             lr=lr)
    elif optimizer == "SGD":
        optimizer = torch.optim.SGD(net.parameters(),
                        lr=lr,
                        momentum=0.9,
                        weight_decay=0.0005)

    # Definition of the loss function ADD MORE IF YOU WANT
    if loss == "MSE":
        criterion = nn.MSELoss()
    elif loss == "MAE":
        criterion = nn.L1Loss()
    elif loss == "CE":
        criterion = CELoss()
    elif loss == "Cross":
        weights = torch.tensor([0.5, 1.0, 2.0])
        weights = weights.to(device)
        criterion = nn.CrossEntropyLoss(weights)
    
    # Saving History to csv
    header = ['epoch', 'train loss', 'validation loss']
    
    time_start = time.time()
    # Run the training and validation
    for epoch in range(epoch_inicial, epochs):
        print('\nStarting epoch {}/{}.'.format(epoch + 1, epochs))

        train_loss, train_acc = train_net(net, device, train_loader, optimizer, criterion, batch_size, loss)
        val_loss, val_acc = val_net(net, device, val_loader, criterion, batch_size)
        
        values = [epoch+1, train_loss, val_loss]
        export_history(header, values, dir_results, "result"+str(run)+".csv")

        save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': net.state_dict(),
                'loss': train_loss,
                'optimizer' : optimizer.state_dict(),
              }, path=dir_results, filename='weights' + str(run) + '.pth')
        
        # save model
        if val_loss < best_loss:
            best_loss = val_loss
            save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': net.state_dict(),
                    'loss': train_loss,
                    'optimizer' : optimizer.state_dict(),
                }, path=dir_results, filename='best_weights' + str(run) + '.pth')

    time_dif = time.time() - time_start
    print("It tooks %.4f seconds to finish the run." % (time_dif))
    
    return train_loss, val_loss

# K-Folds Cross Validation
As the idea is to make a "end to end" K-Fold process over the whole pipeline, we need to make the difference between the proccess for the back and top model. \\
The main difference being that in the back model we create the partitions while in the top model we should only be using the same partitions. \\
Therefore, the idea here is to export each partition of ids with each loader in the back model.

## K-Folds Setup

In [0]:
# Function to do a training with Cross Validation for Base Model
def Kfolds_validation_single(k, # Amount of K Folds to do
                          epoch_inicial,  # Epoch to start the training from - Useful as time in colab is limited
                          iteracion_inicial, # Iteration from the k folds to start training from - Same as above
                          batch_size, 
                          dir_img,
                          dir_gt, 
                          dir_predictions, 
                          dir_results):
  
  # Obtain the loaders for validation
  ids = get_partitions(k)

  # Create folders for Back and Top checkpoints
  if not os.path.isdir(dir_results):
    os.mkdir(dir_results)

  for i in range(iteracion_inicial, k):

    # We should be loading weights only if the initial epoch is a value over 0
    if epoch_inicial != 0:
        load = dir_results + "weights" + str(i) + ".pth"
    else:
        load = None

    # Obtains loaders for i-th iteration of kfolds cross validation for back model
    back_loaders = get_loaders_kfolds(ids, k, dir_img, dir_gt, batch_size, i, None, None)
      
    # Runs the training from a certain point in time and for the back model
    train_loss, test_loss = setup_and_run_train(n_channels = 1, 
                                                n_classes = 3, 
                                                dir_img = dir_img,
                                                dir_gt = dir_gt, 
                                                dir_results = dir_results, 
                                                load = load, 
                                                val_perc = 0, 
                                                batch_size = 10, 
                                                epoch_inicial = epoch_inicial,
                                                epochs = 50,
                                                lr = 0.001, 
                                                run = i, 
                                                optimizer = "Adam", 
                                                loss = "CE", 
                                                loaders = back_loaders)

    # Restarts the epoch_inicial variable so that the next fold we start from the epoch 0
    epoch_inicial = 0

    print("\nK-Folds {} iteration completed \n".format(i))
  
  print("K-Folds training completed")


In [0]:
# Function to do a training with Cross Validation with E2E (End-to-End)
def Kfolds_validation_E2E(k, # Amount of K Folds to do
                          epoch_inicial,  # Epoch to start the training from - Useful as time in colab is limited
                          iteracion_inicial, # Iteration from the k folds to start training from - Same as above
                          back, # True if the back model was being trained (If traning needs to start from back) - Same as above
                          two_inputs, # Variable to define if training should be done merging the input and DDT prediction
                          batch_size, 
                          dir_img,
                          dir_gt, 
                          dir_gt2, 
                          dir_predictions, 
                          dir_results):
  
  # Obtain the loaders for validation
  ids = get_partitions(k)

  # When training with "two inputs" the amount of channels change to 2 on top_model
  if two_inputs:
    top_n_channels = 2
  else:
    top_n_channels = 1

  results_back = dir_results + "Back/"
  results_top = dir_results + "Top/"

  # Create folders for Back and Top checkpoints
  if not os.path.isdir(results_back):
    os.makedirs(results_back)
  if not os.path.isdir(results_top):
    os.makedirs(results_top)

  for i in range(iteracion_inicial, k):
    # If the back-model needs to be trained
    if back:
      
      # We should be loading weights only if the initial epoch is a value over 0
      if epoch_inicial != 0:
        load = results_back + "weights" + str(i) + ".pth"
      else:
        load = None

      # Obtains loaders for i-th iteration of kfolds cross validation for back model
      back_loaders = get_loaders_kfolds(ids, k, dir_img, dir_gt, batch_size, i, None, None)
      
      # Runs the training from a certain point in time and for the back model
      train_loss, test_loss = setup_and_run_train(n_channels = 1, 
                                                  n_classes = 1, 
                                                  dir_img = dir_img,
                                                  dir_gt = dir_gt, 
                                                  dir_results = results_back, 
                                                  load = load, 
                                                  val_perc = 0, 
                                                  batch_size = 10, 
                                                  epoch_inicial = epoch_inicial,
                                                  epochs = 50,
                                                  lr = 0.001, 
                                                  run = i, 
                                                  optimizer = "Adam", 
                                                  loss = "MAE", 
                                                  loaders = back_loaders)
      
        
      print("\nBack Model training completed\n")
      
      # Resets the epoch_inicial variable so that the top model starts from the epoch 0
      epoch_inicial = 0

    # We will be running the Top Model always

    # We should be loading weights only if the initial epoch is a value over 0
    if epoch_inicial != 0:
      load = results_top + "weights" + str(i) + ".pth"
    else:
      load = None
    
    # Obtains loaders for i-th iteration of kfolds cross validation for top model
    back_weights_path = results_back + "best_weights" + str(i) + ".pth"
    top_loaders = get_loaders_kfolds(ids, k, dir_img, dir_gt2, batch_size, i, back_weights_path, two_inputs)
    
    # Runs the training from a certain point in time and for the top model
    train_loss, test_loss = setup_and_run_train(n_channels = top_n_channels, 
                                                n_classes = 3, 
                                                dir_img = "Predicted from Back-Model",
                                                dir_gt = dir_gt2, 
                                                dir_results = results_top, 
                                                load = load, 
                                                val_perc = 0, 
                                                batch_size = 10,
                                                epoch_inicial = epoch_inicial,
                                                epochs = 50,
                                                lr = 0.001, 
                                                run = i, 
                                                optimizer = "Adam", 
                                                loss = "CE", 
                                                loaders = top_loaders)
    
    print("\nTop Model training completed\n")

    # Restarts the epoch_inicial variable so that the next fold we start from the epoch 0
    epoch_inicial = 0

    # Regardless, the back model should always be runned on the next iteration
    back = True

    print("\nK-Folds {} iteration completed \n".format(i))
  
  print("K-Folds training completed")


## K-Folds End To End Training

In [0]:
#For DNLM03 ONLY
#globalMean = 0.1420;
#globalStd = 0.1082;

Kfolds_validation_E2E(k=5, # Amount of K Folds to do
                      epoch_inicial=39,  # Epoch to start the training from - Useful as time in colab is limited
                      iteracion_inicial=4, # Iteration from the k folds to start training from - Useful as time in colab is limited
                      back=False, # True if the back model was being trained - Useful as time in colab is limited
                      two_inputs=False,
                      batch_size=10, 
                      dir_img = 'DNLM03/',
                      dir_gt='DTLabels/', 
                      dir_gt2='BTLabels/', 
                      dir_predictions="predictions/DNLM03/", 
                      dir_results='checkpoints/DNLM03/')

## K-Folds Base Model Training

In [0]:
Kfolds_validation_single(k=0, # Amount of K Folds to do
                      epoch_inicial=0,  # Epoch to start the training from - Useful as time in colab is limited
                      iteracion_inicial=0, # Iteration from the k folds to start training from - Useful as time in colab is limited
                      batch_size=10, 
                      dir_img = 'Original/',
                      dir_gt='BTLabels/', 
                      dir_predictions="predictions/BaseModel/", 
                      dir_results='checkpoints/BaseModel/')

## K-Folds Prediction

In [0]:
# Saves the dataset of every fold
k = 5
for i in range(0, k):
  save_predictions(i=i,
                   k=k,
                   dir_predictions="predictions/BaseModel/", 
                   results_top='checkpoints/BaseModel/', 
                   results_back=None,
                   dir_img='Original/',
                   dir_gt='BTLabels/',
                   two_inputs=False)
  save_predictions(i=i,
                   k=k, 
                   dir_predictions="predictions/E2E/", 
                   results_top='checkpoints/E2E/Top/', 
                   results_back='checkpoints/E2E/Back/',
                   dir_img='Original/',
                   dir_gt='BTLabels/',
                   two_inputs=False)
  save_predictions(i=i,
                   k=k, 
                   dir_predictions="predictions/TwoInputE2E/", 
                   results_top='checkpoints/TwoInputE2E/Top/', 
                   results_back='checkpoints/TwoInputE2E/Back/',
                   dir_img='Original/',
                   dir_gt='BTLabels/',
                   two_inputs=True)
  
  